In [1]:
import json
import ast

import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from tqdm.auto import tqdm
from scipy.stats import levene
import scipy.stats as st
import pylab
import statsmodels.stats.api as sms
import phik

import my_hypotheses as hyp
import LichessAnalys as li
import my_message as ms

c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lichessAnalys = li.LichessAnalys()
hypotheses = hyp.ProgressivePlayerCanBeACheater()
message_to_send = ms.MessageToSend()

In [20]:
df_for_test_group = pd.read_csv('./move_score_and_clocks.csv', sep=',')

In [21]:
df_for_control_group = \
     pd.read_csv('./merge_eval_and_clocks_for_control_group.csv', sep=',')

In [22]:
df_for_test_group.head()

,date,game_id,time_control,clocks_list,move_count,user_id,move_score
0,2023-05-15,BHshU6Ph,30+0,"[1.76, 2.32, 1.76, 2.0, 7.28, 1.92, 1.92, 3.44...",52,alex_gor001,"[23, 12, 6, 8, 31, 0, 0, -10, -10, 15, -17, -1..."
1,2023-05-15,Ry1UtZeg,30+0,"[1.84, 0.56, 1.04, 2.08, 1.84, 9.52, 18.48, 7....",32,alex_gor001,"[0, 29, 39, 39, 35, 25, 36, -9, 54, 51, 11, 37..."
2,2023-05-10,F9DKP94I,30+0,"[3.12, 2.8, 1.52, 1.84, 9.04, 8.72, 6.88, 3.2,...",38,alex_gor001,"[0, 16, 22, 31, 12, 0, 0, 75, -18, -120, 0, 0,..."
3,2023-05-09,mXJiUKxR,30+0,"[1.2, 3.04, 1.76, 1.2, 1.52, 2.4, 2.8, 2.56, 6...",29,alex_gor001,"[36, 66, 80, 48, 98, 79, 97, 86, 88, 118, 140,..."
4,2023-05-09,Z5YQlFc3,30+0,"[2.72, 2.16, 3.84, 2.32, 5.12, 3.76, 1.6, 3.52...",41,alex_gor001,"[0, 29, 32, 24, 12, 47, 60, 64, 63, 58, 14, 77..."


In [29]:
def add_correlation_coefficient(
    df_for_satatistical_test: pd.DataFrame) -> pd.Series:
    df_for_satatistical_test['clocks_list_new'] = \
    [json.loads(i) for i in df_for_satatistical_test['clocks_list']]
    df_for_satatistical_test['move_score_new'] = \
    [json.loads(i) for i in df_for_satatistical_test['move_score']]

    len_df = df_for_satatistical_test.shape[0]
    list_df_len = list(range(0, len_df, 1))
    correlation_list = []

    for i in list_df_len:
        len_clocks = len(df_for_satatistical_test['clocks_list_new'][i])
        len_score = len(df_for_satatistical_test['move_score_new'][i])
        if len_clocks != len_score: 
            if len_clocks > len_score:
                df_for_satatistical_test['clocks_list_new'][i].pop(-1)
            else:
                df_for_satatistical_test['move_score_new'][i].pop(-1)
            d = {
            'clocks_list': df_for_satatistical_test['clocks_list_new'][i],
            'move_score': df_for_satatistical_test['move_score_new'][i]
            }
            df = pd.DataFrame(data=d)
            t = df[['clocks_list', 'move_score']]
            phik_overview = t.phik_matrix()
            correlation_coefficient = phik_overview['clocks_list'][1]
            correlation_list.append(correlation_coefficient)
        else:
            d = {
            'clocks_list': df_for_satatistical_test['clocks_list_new'][i],
            'move_score': df_for_satatistical_test['move_score_new'][i]
            }
            df = pd.DataFrame(data=d)
            t = df[['clocks_list', 'move_score']]
            phik_overview = t.phik_matrix()
            correlation_coefficient = round(phik_overview['clocks_list'][1], 2)
            correlation_list.append(correlation_coefficient)
    result = pd.Series(correlation_list)
    return result

In [30]:
add_correlation_coefficient = add_correlation_coefficient(
        df_for_satatistical_test=df_for_test_group)
add_correlation_coefficient \
    .to_csv('./add_correlation_coefficient.csv',
    index=False,
    sep=",")

interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval columns not set, guessing: ['clocks_list', 'move_score']
interval c

In [31]:
print(add_correlation_coefficient)

0     0.543718
1     0.610000
2     0.805791
3     0.000000
4     0.000000
5     0.340000
6     0.627993
7     0.260000
8     0.470000
9     0.110000
10    0.340000
11    0.090000
12    0.800000
13    0.000000
14    0.460000
15    0.000000
16    0.000000
17    0.000000
dtype: float64


In [9]:
def levene_test(df_for_control_group: pd.DataFrame,
    df_for_test_group: pd.DataFrame) -> pd.DataFrame:
    result = pd.DataFrame(columns=[
            'levene_p_value_list'])

    df_for_test_group['clocks_list_new'] = \
    [ast.literal_eval(i) for i in df_for_test_group['clocks_list']]
    df_for_control_group['clocks_list_new'] = \
    [ast.literal_eval(i) for i in df_for_control_group['clocks_list']]

    test_group = df_for_test_group['clocks_list_new']
    control_group = df_for_control_group['clocks_list_new']

    for i in test_group:
        levene_p_value_list = []
        for k in control_group:
            stat, p = levene(k, i, center='mean')
            levene_p_value_list.append(round(p, 3))
        columns = {'levene_p_value_list': str(levene_p_value_list)}
        df = pd.DataFrame(data=columns, index=[0])
        result = \
        pd.concat([result, \
            df], ignore_index=True , join="outer")

    result['levene_p_value_list'] = \
    [ast.literal_eval(i) for i in result['levene_p_value_list']]

    leven_median = []
    for i in result['levene_p_value_list']:
        leven_median.append(np.median(i))
    result['levene_p_value_median'] = leven_median
    result['game_id'] = df_for_test_group['game_id']
    result = result.drop(columns=['levene_p_value_list'])
    return result

In [10]:
levene_test = levene_test(
    df_for_control_group=df_for_control_group,
    df_for_test_group=df_for_test_group)
levene_test \
    .to_csv('./levene_test.csv',
    index=False,
    sep=",")

In [11]:
lev = pd.read_csv('./levene_test.csv',
    sep=",")
lev


,levene_p_value_median,game_id
0,0.000,BHshU6Ph
1,0.041,Ry1UtZeg
2,0.001,F9DKP94I
3,0.027,mXJiUKxR
4,0.001,Z5YQlFc3
5,0.003,qwsQrE0T
6,0.009,9S6uCtUw
7,0.028,PUkXShmF
8,0.016,tCXGKrBo
9,0.033,2ShEjdoO


In [12]:
user_id = 'alex_gor001'

In [13]:
def combining_main_and_statistical_data(
    user_id: str,
    levene_test: pd.DataFrame,
    merge_eval_and_clocks: pd.DataFrame,
    add_correlation_coefficient: pd.Series) -> pd.DataFrame:
    user_id = user_id
    df = merge_eval_and_clocks.query('user_id == @user_id')
    levene_test['correlation_coefficient']= add_correlation_coefficient
    result = df.merge(levene_test,
            on='game_id',
            how='left')
    result = result.dropna()
    return result

In [14]:
df_classical = pd.read_csv('./df_classical.csv', sep=',')

In [15]:
combining_main_and_statistical_data = combining_main_and_statistical_data(
    user_id=user_id,
    levene_test=levene_test,
    merge_eval_and_clocks=df_classical,
    add_correlation_coefficient=add_correlation_coefficient)
combining_main_and_statistical_data \
    .to_csv('./combining_main_and_statistical_data.csv',
    index=False,
    sep=",")

In [16]:
combining_main_and_statistical_data

,game_id,date,game_speed,rating,rating_diff,clocks_mean,clocks_std,clocks_median,move_count,time_control,user_id,mistake,blunder,inaccuracy,acpl,levene_p_value_median,correlation_coefficient
0,BHshU6Ph,2023-05-15,classical,2286,-11,5.62,8.14,2.0,52,30+0,alex_gor001,0.0,1.0,2.0,22.0,0.000,0.543718
1,Ry1UtZeg,2023-05-15,classical,2275,11,13.88,23.56,5.0,32,30+0,alex_gor001,2.0,1.0,4.0,55.0,0.041,0.610000
2,Ry1UtZeg,2023-05-15,classical,2275,11,13.88,23.56,5.0,32,30+0,alex_gor001,0.0,2.0,4.0,32.0,0.041,0.610000
3,F9DKP94I,2023-05-10,classical,2251,7,34.63,53.62,6.5,38,30+0,alex_gor001,3.0,0.0,3.0,26.0,0.001,0.805791
5,mXJiUKxR,2023-05-09,classical,2220,11,7.97,12.67,3.0,29,30+0,alex_gor001,0.0,0.0,0.0,8.0,0.027,0.000000
7,Z5YQlFc3,2023-05-09,classical,2214,9,7.73,7.22,5.0,41,30+0,alex_gor001,1.0,3.0,2.0,36.0,0.001,0.000000
8,qwsQrE0T,2023-05-09,classical,2189,9,9.44,9.20,4.0,41,30+0,alex_gor001,0.0,0.0,3.0,21.0,0.003,0.340000
9,9S6uCtUw,2023-05-09,classical,2168,9,9.42,11.54,3.0,38,30+0,alex_gor001,0.0,0.0,0.0,13.0,0.009,0.627993
10,PUkXShmF,2023-05-08,classical,2157,4,23.15,38.64,6.0,41,25+3,alex_gor001,0.0,4.0,3.0,45.0,0.028,0.260000
11,tCXGKrBo,2023-05-08,classical,2159,-2,11.22,23.73,1.0,60,25+3,alex_gor001,0.0,0.0,4.0,12.0,0.016,0.470000
